# 📊 Telecom X - Predição de Evasão de Clientes (Parte 2)

Este notebook contém a **Parte 2** do desafio da Telecom X: construção de modelos preditivos para prever a evasão de clientes.

---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings('ignore')

## 1. Carregar os dados tratados

In [ ]:
df = pd.read_csv('TelecomX_limpo.csv')
df.head()

## 2. Pré-processamento dos dados

In [ ]:
# Remover colunas irrelevantes
if 'customerID' in df.columns:
    df = df.drop(columns=['customerID'])

# One-hot encoding para variáveis categóricas
df = pd.get_dummies(df, drop_first=True)

df.head()

## 3. Balanceamento das classes

In [ ]:
X = df.drop('Churn_Yes', axis=1)
y = df['Churn_Yes']

print('Proporção de classes:')
print(y.value_counts(normalize=True))

# Aplicar SMOTE
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)

print('\nApós SMOTE:')
print(y_res.value_counts(normalize=True))

## 4. Divisão treino/teste

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.3, random_state=42, stratify=y_res)
X_train.shape, X_test.shape

## 5. Modelagem

Treinaremos dois modelos:
- Regressão Logística (com normalização)
- Random Forest (sem normalização)

In [ ]:
# Normalização para modelos baseados em distância
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Regressão Logística
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train_scaled, y_train)
y_pred_log = logreg.predict(X_test_scaled)

# Random Forest
rf = RandomForestClassifier(random_state=42, n_estimators=200)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

## 6. Avaliação dos modelos

In [ ]:
print('--- Regressão Logística ---')
print(classification_report(y_test, y_pred_log))
ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred_log)).plot()
plt.show()

print('--- Random Forest ---')
print(classification_report(y_test, y_pred_rf))
ConfusionMatrixDisplay(confusion_matrix(y_test, y_pred_rf)).plot()
plt.show()

## 7. Importância das variáveis

In [ ]:
importances = pd.Series(rf.feature_importances_, index=X_train.columns)
importances.sort_values(ascending=False).head(10).plot(kind='bar', figsize=(10,5))
plt.title('Top 10 Variáveis mais importantes - Random Forest')
plt.show()